In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e5/sample_submission.csv
/kaggle/input/playground-series-s3e5/train.csv
/kaggle/input/playground-series-s3e5/test.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import cohen_kappa_score

In [3]:
train=pd.read_csv('/kaggle/input/playground-series-s3e5/train.csv')
sub=pd.read_csv('/kaggle/input/playground-series-s3e5/sample_submission.csv')
test=pd.read_csv('/kaggle/input/playground-series-s3e5/test.csv')

## Data normalization

In [4]:
y=train.pop('quality')
x=train.drop('Id',axis=1)

In [5]:
# min max normalization
def min_max_normalization(df):
    return (df - df.min()) /(df.max()-df.min())
x=min_max_normalization(x)
x.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,0.275229,0.228571,0.513158,0.078125,0.151741,0.432836,0.113475,0.414831,0.567308,0.243590,0.641509
1,0.394495,0.085714,0.960526,0.085937,0.199005,0.432836,0.212766,0.621880,0.557692,0.179487,0.773585
2,0.192661,0.235714,0.039474,0.070312,0.116915,0.029851,0.017730,0.479442,0.750000,0.217949,0.490566
3,0.284404,0.492857,0.289474,0.109375,0.179104,0.149254,0.205674,0.530837,0.442308,0.089744,0.207547
4,0.321101,0.128571,0.394737,0.085937,0.166667,0.134328,0.134752,0.320852,0.442308,0.621795,0.150943


In [6]:
# # z_score normaliation
# def z_score_normalization(df):
#     return (df - df.mean()) /df.std()
# x=z_score_normalization(x)
# x.head()

# Model

In [7]:
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.model_selection import GridSearchCV
# train_x,val_x,train_y,val_y=train_test_split(x,y,train_size=0.75,random_state=1)
# # Define the parameter grid to search
# param_grid = {'loss':['log_loss','deviance','exponential'],
#               'learning_rate': [0.1, 0.05, 0.01],
#               'n_estimators': [50,75,100, 200],
#               'max_depth': [1, 3, 5,8,10],
#              'criterion':['friedman_mse','squared_error'],
#               'max_features':['auto','sqrt','log2'],
#              'ccp_alpha':[0,0.001,0.01]}

# # Initialize the GradientBoostingClassifier
# gbc = GradientBoostingClassifier()

# # Use GridSearchCV to find the best hyperparameters for the classifier
# grid_search = GridSearchCV(gbc, param_grid, cv=5)
# grid_search.fit(train_x, train_y)

# # Print the best hyperparameters found by GridSearchCV
# print("Best hyperparameters: ", grid_search.best_params_)


## GradiantBoostClassifier

optimized_hyperparametertuning:

Best hyperparameters:  {'ccp_alpha': 0.001, 'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 200}

In [8]:
train_x,val_x,train_y,val_y=train_test_split(x,y,train_size=0.75,random_state=1)
model=GradientBoostingClassifier(criterion='squared_error',learning_rate=0.1, loss='deviance',
                              max_depth=2,max_features='sqrt',n_estimators=50, random_state=1,ccp_alpha=0.001)
model.fit(train_x,train_y)
y_pred=model.predict(train_x)
val_pred=model.predict(val_x)
score_train=cohen_kappa_score(y_pred,train_y)
score_val=cohen_kappa_score(val_pred,val_y)
print(score_train,score_val)

0.391415057298957 0.3320016365430435


In [9]:
test_x=test.iloc[:,1:]
test_x=min_max_normalization(test_x)
test_x.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,0.214953,0.322314,0.013158,0.100000,0.142857,0.447761,0.292994,0.564356,0.511811,0.162602,0.192308
1,0.214953,0.524793,0.197368,0.100000,0.197802,0.194030,0.171975,0.440975,0.464567,0.235772,0.230769
2,0.327103,0.280992,0.526316,0.100000,0.116484,0.298507,0.267516,0.584158,0.267717,0.211382,0.134615
3,0.289720,0.289256,0.526316,0.071429,0.096703,0.194030,0.108280,0.360244,0.440945,0.211382,0.557692
4,0.149533,0.181818,0.421053,0.114286,0.070330,0.104478,0.547771,0.265804,0.385827,0.073171,0.769231


In [10]:
#model.fit(x,y)
y_pred=model.predict(test_x)
submission=pd.DataFrame({'Id':test['Id'],'quality':y_pred})
submission.to_csv('submission.csv',index=False)